In [1]:
import torch

In [4]:
a = torch.rand(128, dtype=torch.float16)
b = torch.rand(128, dtype=torch.float16)

In [5]:
c1 = a*b
c2 = a.to(torch.float32) * b.to(torch.float32)

In [6]:
c1

tensor([0.3093, 0.1486, 0.2263, 0.8916, 0.3855, 0.0579, 0.0817, 0.5801, 0.3835,
        0.1164, 0.2279, 0.6621, 0.2598, 0.1498, 0.3154, 0.2615, 0.1338, 0.0113,
        0.1447, 0.6582, 0.1236, 0.1976, 0.1465, 0.3062, 0.0307, 0.5923, 0.0487,
        0.0110, 0.2064, 0.1727, 0.1053, 0.1340, 0.8179, 0.0706, 0.1030, 0.1567,
        0.0762, 0.3010, 0.5771, 0.4688, 0.0794, 0.3213, 0.1927, 0.0941, 0.1847,
        0.2942, 0.1024, 0.0995, 0.1141, 0.6401, 0.0090, 0.0830, 0.1732, 0.1412,
        0.4048, 0.0495, 0.0788, 0.3359, 0.0850, 0.6787, 0.4414, 0.0462, 0.0737,
        0.0154, 0.0302, 0.5249, 0.5952, 0.5352, 0.0052, 0.0266, 0.0911, 0.2563,
        0.8184, 0.4243, 0.0065, 0.4751, 0.2367, 0.3052, 0.3474, 0.1927, 0.7285,
        0.3162, 0.2551, 0.1517, 0.5142, 0.0115, 0.1653, 0.1229, 0.3940, 0.0090,
        0.5332, 0.2117, 0.0290, 0.6899, 0.2302, 0.3787, 0.5771, 0.0144, 0.5547,
        0.1182, 0.6401, 0.0689, 0.1064, 0.0612, 0.0108, 0.0050, 0.3208, 0.6113,
        0.1385, 0.2104, 0.0164, 0.3665, 

In [7]:
c2

tensor([0.3092, 0.1485, 0.2263, 0.8917, 0.3855, 0.0579, 0.0817, 0.5802, 0.3836,
        0.1164, 0.2280, 0.6620, 0.2599, 0.1497, 0.3154, 0.2615, 0.1338, 0.0113,
        0.1447, 0.6583, 0.1236, 0.1977, 0.1465, 0.3062, 0.0308, 0.5924, 0.0487,
        0.0110, 0.2064, 0.1727, 0.1054, 0.1340, 0.8181, 0.0706, 0.1030, 0.1567,
        0.0762, 0.3011, 0.5770, 0.4688, 0.0794, 0.3214, 0.1927, 0.0941, 0.1847,
        0.2942, 0.1024, 0.0995, 0.1142, 0.6400, 0.0090, 0.0830, 0.1733, 0.1412,
        0.4047, 0.0495, 0.0788, 0.3360, 0.0849, 0.6785, 0.4414, 0.0462, 0.0736,
        0.0154, 0.0302, 0.5247, 0.5952, 0.5353, 0.0052, 0.0266, 0.0910, 0.2563,
        0.8186, 0.4243, 0.0065, 0.4750, 0.2367, 0.3051, 0.3474, 0.1928, 0.7287,
        0.3161, 0.2551, 0.1517, 0.5141, 0.0115, 0.1653, 0.1230, 0.3939, 0.0090,
        0.5332, 0.2117, 0.0290, 0.6901, 0.2303, 0.3786, 0.5772, 0.0144, 0.5545,
        0.1182, 0.6402, 0.0689, 0.1064, 0.0612, 0.0108, 0.0050, 0.3207, 0.6111,
        0.1386, 0.2104, 0.0164, 0.3665, 